# Notebook to play with SMAs

Latest version: 2024-08-16  
Author: MvS

## Description


## Result




In [ ]:
import yfinance as yf
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from importlib import reload  # Python 3.4+
import utils.indicator_utils as idc

import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO, format="%(asctime)s %(message)s")

periods = [200, 50]

dt_end = datetime.datetime.today()
dt_data_start = dt_end - datetime.timedelta(days=max(periods) * 3)

# stock = "FROG"
# stock = "TSLA"
# stock = "NVDA"
stock = "SAP.TO"

try:
    # Grab sufficient stock data for averaging SMAs
    load_df = yf.download(
        f"{stock}",
        start=dt_data_start.strftime('%Y-%m-%d'),
        end=dt_end.strftime('%Y-%m-%d'),
        progress=False,
    )

    assert load_df.shape[1] == 6 and load_df.shape[0] > max(periods)

except AssertionError:
    logging.info(f"Download failed for symbol {stock}.  Skipping...")


In [ ]:
sym_df = load_df.copy()

# Compute the simple moving average (SMA)
for period in periods:
    sym_df[f"SMA_{period:03}"] = idc.sma(sym_df['Close'], length=period)

# Now that we calculated the SMA, we can remove the data points before the actual AOI that we want.
sym_df = sym_df[-max(periods) :].copy()


dt_start = sym_df.index[0]

logging.info(
    f"""Calculating SMAs for {periods} length starting from:
  {dt_start.strftime('%Y-%m-%d')} and ending on {dt_end.strftime('%Y-%m-%d')}
  and containing {sym_df.shape} data points."""
)

sym_df[[f"SMA_{min(periods):03}", f"SMA_{max(periods):03}", 'Close']].plot()

In [ ]:
# del sym_df

In [ ]:
ema_length = 30
ema_scaler = 1.01

long_ema_str = f"EMA_{ema_length:03}_Long"
short_ema_str = f"EMA_{ema_length:03}_Short"
min_sma_str = f"SMA_{min(periods):03}"
max_sma_str = f"SMA_{max(periods):03}"

# STOP LOSS LONG
sym_df[long_ema_str] = idc.ema(sym_df['Low'], length=ema_length) / ema_scaler

# STOP LOSS SHORT
sym_df[short_ema_str] = idc.ema(sym_df['High'], length=ema_length) * ema_scaler

sym_df[
    [
        min_sma_str,
        max_sma_str,
        'Close',
        long_ema_str,
        short_ema_str,
    ]
].plot()


logging.info(f"{min_sma_str:>14} {sym_df['SMA_050'][-1:].mean():8.3f}")
logging.info(f"{max_sma_str:>14} {sym_df['SMA_200'][-1:].mean():8.3f}")
logging.info(f"{'Close:':>14} {sym_df['Close'][-1:].mean():8.3f}")
logging.info(f"{long_ema_str:>14} {sym_df['EMA_030_Long'][-1:].mean():8.3f}")
logging.info(f"{short_ema_str:>14} {sym_df['EMA_030_Short'][-1:].mean():8.3f}")

In [ ]:
sym_df

In [ ]:
reload(idc)

# Clean up
sym_df.drop(
    ['Mode', 'P_Signal', 'N_Signal', 'Buy', 'Sell',],
    axis=1,
    inplace=True,
    errors='ignore',
)

# Generate crossover signals
temp = idc.crossovers(short=sym_df[min_sma_str], long=sym_df[max_sma_str], scaledsignal=False)
sym_df = pd.concat([sym_df, temp], axis=1)

# Re-label columns
sym_df = sym_df.rename(columns={'P_Signal': 'Buy', 'N_Signal': 'Sell'})
del temp



# Define the columns and their styles
columns = [
    'Close',
    min_sma_str,
    max_sma_str,
    'Mode',
    'Buy',
    'Sell',
]
colors = ['grey', 'darkgreen', 'orangered', 'black', 'darkgreen', 'orangered']
linestyles = ['-', '-', '-', ':', None, None]
markers = [None, None, None, None, '^', 'v']


try:
    # Plot each column separately
    ax = None
    for col, color, style, marker in zip(columns, colors, linestyles, markers):
        ax = sym_df[col].plot(color=color, linestyle=style, marker=marker, ax=ax)
    ax.legend(loc='upper left')
except TypeError:
    logging.info(f"Either no BUY/SELL signal found")